**Pulling Data from Dartfish**

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException, TimeoutException

# Specify the path to  your download folder
download_folder = '/Users/lkimball/Desktop/Wesleyan_Tennis_Analytics/Match_csvs'

# Create ChromeOptions
options = Options()
options.add_experimental_option('prefs', {
    "download.default_directory": download_folder,  # Specify the download folder
    "download.prompt_for_download": False,  # Disable download prompt
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})



# Specify the path to ChromeDriver and create a Service object
service = Service(executable_path='/Users/lkimball/Desktop/chromedriver_mac_x64/chromedriver')

# Initialize the Chrome WebDriver with the specified service and options
driver = webdriver.Chrome(service=service, options=options)

# Replace 'login_page_url' with the actual login page URL
driver.get('https://www.dartfish.tv/Login')

# Replace 'your_username' and 'your_password' with your actual login credentials
username = 'lucaskimball98@gmail.com'
password = 'LUCASgjoa13!'

# Locate the username input field by its ID and enter the username
username_field = driver.find_element(By.ID, 'ctl00_MC_Usn')
username_field.send_keys(username)  # Replace 'your_username' with your actual username


# Locate the password input field by its ID and enter the password
password_field = driver.find_element(By.ID, 'ctl00_MC_PwdReader_Password')
password_field.send_keys(password)  # Replace 'your_password' with your actual password


# Locate the login button by its ID and click it
login_button = driver.find_element(By.ID, 'ctl00_MC_LoginButton')

# Option 1: Use WebDriver's click function
try:
    login_button.click()
except Exception as e:
    # If the regular click doesn't work, fallback to JavaScript click
    driver.execute_script("arguments[0].click();", login_button)


# Wait for the login process to complete, adjust the waiting time as necessary
time.sleep(5)


driver.get('https://www.dartfish.tv/Videos?CR=p98196')


# Wait for the dropdown to be available and interactable
dropdown = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "ctl00_MCL_MLister_PaginationControl_ItemsPerPageBottom_ItemCountDD")))

# Initialize a Select object with the dropdown
select = Select(dropdown)

# Select the option for 200 items per page by value
select.select_by_value('200')

# Wait for the page to reload/update with 200 items displayed per page, adjust the waiting time as needed
time.sleep(5)  # Adjust the waiting time as needed based on how long the page takes to refresh



# Assuming you're already on the page with the matches


processed_matches = set()

try:
    overlay_dismiss_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button[title='Accept'], button[aria-label='Accept']"))
    )
    overlay_dismiss_button.click()
    print("Overlay dismissed.")
except TimeoutException:
    print("No overlay found or it's not clickable.")

matches = driver.find_elements(By.CSS_SELECTOR, "a[href^='Player?CR=']")
print(f"Total matches found: {len(matches)}")

for index in range(len(matches)):
    matches = driver.find_elements(By.CSS_SELECTOR, "a[href^='Player?CR=']")
    match = matches[index]
    match_url = match.get_attribute('href')

    if match_url in processed_matches:
        print(f"Match at index {index} already processed.")
        continue

    processed_matches.add(match_url)
    print(f"Processing match at index {index}: {match_url}")

    try:
        match.click()
    except ElementClickInterceptedException:
        driver.execute_script("arguments[0].scrollIntoView(true); arguments[0].click();", match)

    try:
        download_excel = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "ctl00_MCL_CsvPaneLink"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", download_excel)
        download_excel.click()
        print("Clicked on Excel download link.")
    except TimeoutException:
        print("Excel download link not clickable or not found.")

    try:
        csv_download_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "ctl00_MCL_ExportFileList_ExportMatch"))
        )
        csv_download_link.click()
        print("Clicked on CSV download link.")
    except TimeoutException:
        print(f"CSV download link not found for match at index {index}.")
        driver.back()
        continue

    time.sleep(2)  # Adjust based on the actual time it takes for downloads to initiate

    driver.back()
    WebDriverWait(driver, 10).until(
        EC.visibility_of_any_elements_located((By.CSS_SELECTOR, "a[href^='Player?CR=']"))
    )

driver.quit()

import pandas as pd
import os

# List all CSV files in the download directory
csv_files = [f for f in os.listdir(download_folder) if f.endswith('.csv')]

# Initialize an empty list to store DataFrames
dfs = []

# Iterate over CSV files, read each into a DataFrame, and append to the list
for file in csv_files:
    df = pd.read_csv(os.path.join(download_folder, file))
    dfs.append(df)

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)


# Optionally, save the combined DataFrame to a CSV file (for verification or local use)
combined_df.to_csv('/Users/lkimball/Desktop/Wesleyan_Tennis_Analytics/combined_file.csv', index=False)

No overlay found or it's not clickable.
Total matches found: 162
Processing match at index 0: https://www.dartfish.tv/Player?CR=p98196c565725m8605931
Clicked on Excel download link.
CSV download link not found for match at index 0.


IndexError: list index out of range

In [1]:
import boto3

# Initialize a boto3 client
s3_client = boto3.client('s3')

bucket_name = 'collegetennismatchdata'  # Your S3 bucket name

s3_key = 'Initial_dataset/combined_data.csv'  # Path within the bucket, including the folder and file name


# Convert the DataFrame to CSV format and upload directly to S3
from io import StringIO
csv_buffer = StringIO()
combined_df.to_csv(csv_buffer, index=False)

# Upload the CSV data to S3
s3_client.put_object(Bucket=bucket_name, Key=s3_key, Body=csv_buffer.getvalue())


NameError: name 'webdriver' is not defined